In [1]:
import pandas as pd
import numpy as np
import os
import csv

df = pd.DataFrame(columns=['Date', 'Visiting Team', 'Home Team' , 'Visiting Score', 'Home Score'])


In [2]:
filenames = [i for i in os.listdir("data")]
filenames = sorted(filenames)[:-1]

#(filenames)

In [3]:
def fileReader(i):
    df = pd.DataFrame(columns=['Date', 'Visiting Team', 'Home Team' , 'Visiting Score', 'Home Score'])
    with open('data\\' + i) as csvfile:
        reader = csv.reader(csvfile, delimiter = ',')
        for row in reader:
            #print(row[0:11])
            newRow = {
                'Date': row[0][:4],
                'Visiting Team' : row[3],
                'Home Team' : row[6],
                'Visiting Score' : int(row[9]),
                'Home Score': int(row[10])
                    }
            #print(newRow)
            df = df.append(newRow, ignore_index=True)
        
    return ReOrientForMasterDataFramePart1( df, teamData(df))


In [4]:
def fileReaderPart3(i):
    df = pd.DataFrame(columns=['Date', 'Visiting Team', 'Home Team' , 'Visiting Score', 'Home Score'])
    with open('data\\' + i) as csvfile:
        reader = csv.reader(csvfile, delimiter = ',')
        for row in reader:
            #print(row[0:11])
            newRow = {
                'Date': row[0][:4],
                'Visiting Team' : row[3],
                'Home Team' : row[6],
                'Visiting Score' : int(row[9]),
                'Home Score': int(row[10])
                    }
            #print(newRow)
            df = df.append(newRow, ignore_index=True)
    return df

In [5]:
def GamesWon(teamName, df):
    
    homeDf = df[df['Home Team'] == teamName]
    awayDf = df[df['Visiting Team'] == teamName]
    
    
    gamesWonHome = pd.DataFrame(homeDf['Home Score'] > homeDf['Visiting Score'] , columns=['Trues'])
    gamesWonAway = pd.DataFrame(awayDf['Visiting Score'] > awayDf['Home Score'] , columns=['Trues'])
    
    return len(gamesWonHome[gamesWonHome['Trues'] == True]) + len(gamesWonAway[gamesWonAway['Trues'] == True] )




In [6]:
def teamData(df):
    teamAndScores = {}
    for i in df['Visiting Team'].unique():
        homeTeam, awayTeam = df[df['Home Team'] == i].sum(), df[df['Visiting Team'] == i].sum()
        RunsAllowed, RunsScored = (homeTeam['Visiting Score'] + awayTeam['Home Score']), (homeTeam['Home Score'] + awayTeam['Visiting Score'])
        expectedWinning = (RunsScored * RunsScored) / ((RunsScored * RunsScored) + (RunsAllowed * RunsAllowed))
        teamAndScores[i] = {
            'Year': df['Date'][0],
            'Expected Winning %' : expectedWinning,
            '# Of Games': len(df[df['Visiting Team'] == i] + df[df['Home Team'] == i]),
            'Expected Games Won': expectedWinning* len(df[df['Visiting Team'] == i] + df[df['Home Team'] == i]),
            'Number Of Games Won': GamesWon(i, df),
            'Deviation': round(float(GamesWon(i, df) -  expectedWinning* len(df[df['Visiting Team'] == i] + df[df['Home Team'] == i])), 1)
        }
    return teamAndScores 

#teamAndScores = teamData(df)

In [7]:

def ReOrientForMasterDataFramePart1(df, teamAndScores = teamData(df)):

    Part1Df = pd.DataFrame(columns=['Date', 'Name', 'Exp. Wins', 'Actual Wins', 'Deviation'])
    for i in teamAndScores.keys():
        
        temp = {
            'Date': teamAndScores[i]['Year'],
            'Name': i,
            'Exp. Wins': teamAndScores[i]['Expected Games Won'],
            'Actual Wins' : teamAndScores[i]['Number Of Games Won'],
            'Deviation': teamAndScores[i]['Deviation']
            }

        Part1Df = Part1Df.append(temp, ignore_index=True)
        Part1Df = Part1Df.sort_values(by='Deviation', ascending=False)

    return Part1Df
    



In [8]:
def part2FramesCalculator(result):
    teamFrames ={} #container for all the team frames

    for i in result['Name'].unique():
        teamFrames[i] = result[result['Name'] == i]
       

    return teamFrames

In [9]:
def part2MasterFrame(teamDict):
    teamRow = {}
    
    for team in teamDict.keys():
        teamRow[team] = {
            'Team': team,
            '# of Years': int(len(teamDict[team]['Name'])),
            'Avg. Wins': round(teamDict[team]['Actual Wins'].mean(), 1),
            'Exp. Avg. Wins': round(teamDict[team]['Exp. Wins'].mean(), 1),
            'Avg. Deviation': round(teamDict[team]['Actual Wins'].mean() - teamDict[team]['Exp. Wins'].mean(), 1)
        }
        
       
    return teamRow

In [10]:
def homeGamesWonPart3(yearDataFrame):
    
    
    gamesWonHome = pd.DataFrame(yearDataFrame['Home Score'] > yearDataFrame['Visiting Score'] , columns=['Trues'])
   
    return len(gamesWonHome[gamesWonHome['Trues'] == True]) /len(yearDataFrame) * 162


In [11]:
def awayGamesWonPart3(yearDataFrame):
    gamesWonAway = pd.DataFrame(yearDataFrame['Home Score'] < yearDataFrame['Visiting Score'] , columns=['Trues'])
   
    return len(gamesWonAway[gamesWonAway['Trues'] == True])/len(yearDataFrame) * 162

In [12]:
def expectedWinsPart3(season):

    runsHomeScored = sum(season['Home Score'])
    runsAwayScored = sum(season['Visiting Score'])
    
    runsHomeScored = round((runsHomeScored **2/((runsHomeScored**2) + (runsAwayScored**2))) * 162, 1)
    runsAwayScored = round((runsAwayScored **2/((sum(season['Home Score'])**2) + (runsAwayScored**2))) * 162, 1) 


    return (runsHomeScored, runsAwayScored)

In [13]:
def Part3MasterFrame(framesPart3):
    teamDictPart3 = {}
    MasterDfPart3 = pd.DataFrame()
    for season in framesPart3:  
       expWins = expectedWinsPart3(season)
       teamDictPart3[season['Date'][0]] = [{
                'Year': season['Date'][0],
                'Team' : 'Home',
                '# Actual Wins': round(homeGamesWonPart3(season), 1),
                '# Expected Wins': expWins[0],
                'Deviation': round(homeGamesWonPart3(season) - expWins[0], 1)
                },{
                'Year': season['Date'][0],
                'Team' : 'Away',
                '# Actual Wins': round(awayGamesWonPart3(season), 1),
                '# Expected Wins': expWins[1],
                'Deviation': round(awayGamesWonPart3(season) - expWins[1], 1)
                }
                ]
                
    for season in teamDictPart3.keys():
        MasterDfPart3 = MasterDfPart3.append(teamDictPart3[season][0], ignore_index=True)
        MasterDfPart3 = MasterDfPart3.append(teamDictPart3[season][1], ignore_index=True)
    
    
    return MasterDfPart3

# Analysis - Part 1 - Great Team Performances
## 1.) Calculates the expected win/loss pct for each an every team in the MLB, from the years 1871-2018.
## 2.) List the top 50 teams performances - measured by how many games the team outperformed (more wins) their expected win loss in the given year.

In [14]:
MasterDf = pd.DataFrame(columns=['Date', 'Name', 'Exp. Wins', 'Actual Wins', 'Deviation'])

## Part 1 DataFrame Construction and Display


In [15]:
frames = []
for i in filenames:
    #print(i)
    frames.append(fileReader(i))

#Part1Master = fileReader('GL2017.txt')
#Part2Master = fileReader('GL1905.txt')
#frames = [Part1Master, Part2Master]
result = pd.concat(frames).sort_values(by='Deviation', ascending=False)
result = result.drop_duplicates()
print(result) 


    Date Name   Exp. Wins Actual Wins  Deviation
0   1905  DET   64.390353          79       14.6
0   2017  SDN   57.341416          71       13.7
0   2016  TEX   81.851488          95       13.1
0   2005  ARI   64.474603          77       12.5
0   1972  NYN   70.961943          83       12.0
..   ...  ...         ...         ...        ...
14  1904  CLE   99.035978          86      -13.0
26  1993  NYN   72.783954          59      -13.8
14  1906  CLE  102.860705          89      -13.9
14  1911  PIT   99.328050          85      -14.3
14  1905  CHN  107.703978          92      -15.7

[2894 rows x 5 columns]


## Part 1 Top 50 

In [16]:
print(result[:51])

    Date Name  Exp. Wins Actual Wins  Deviation
0   1905  DET  64.390353          79       14.6
0   2017  SDN  57.341416          71       13.7
0   2016  TEX  81.851488          95       13.1
0   2005  ARI  64.474603          77       12.5
0   1972  NYN  70.961943          83       12.0
0   2018  SEA  77.034080          89       12.0
0   1943  BSN  55.998918          68       12.0
0   1955  KC1  51.005340          63       12.0
0   1984  NYN  78.073245          90       11.9
0   1974  SDN  48.304034          60       11.7
0   2004  NYA  89.433326         101       11.6
0   2008  ANA  88.518618         100       11.5
0   2012  BAL  81.800263          93       11.2
0   2007  ARI  78.756663          90       11.2
0   1954  BRO  80.852658          92       11.1
1   2016  PHI  59.937597          71       11.1
0   1924  BRO  81.188873          92       10.8
0   1970  CIN  91.415379         102       10.6
0   2009  SEA  74.685300          85       10.3
19  1963  HOU  55.814011          66    

# Analysis - Part 2 - Historically Lucky and Unlucky Teams
## 1.) Calculate the average deviation (in games) between actual performance and expected performance across all years for each team
## 2.) Lists top 10 overperforming teams (actually won more games than expected)
## 3.) Lists top 10 underperforming teams (actually won less games than expected).

In [17]:
teamDict = part2MasterFrame(part2FramesCalculator(result))
MasterDfPart2 = pd.DataFrame()
for i in teamDict.keys():
    MasterDfPart2 = MasterDfPart2.append(teamDict[i], ignore_index= True)

In [18]:
MasterDfPart2 = MasterDfPart2[['Team', '# of Years', 'Avg. Wins', 'Exp. Avg. Wins', 'Avg. Deviation']]


In [19]:

bottom10 = MasterDfPart2[MasterDfPart2['Avg. Wins'] < MasterDfPart2['Exp. Avg. Wins']]

## Top 10 Performing Teams

In [20]:
top10 = MasterDfPart2[MasterDfPart2['Avg. Wins'] > MasterDfPart2['Exp. Avg. Wins']]
print(top10.sort_values(by='Avg. Deviation', ascending=False)[:11])

   Team  # of Years  Avg. Wins  Exp. Avg. Wins  Avg. Deviation
46  NY3         1.0       21.0            16.2             4.8
49  BRP         1.0       76.0            71.4             4.6
61  SR1         1.0       22.0            19.0             3.0
62  BL5         1.0       19.0            16.2             2.8
45  BUF         2.0       77.0            74.4             2.6
68  WS6         1.0        5.0             2.6             2.4
67  FW1         1.0        7.0             4.6             2.4
69  SL2         1.0       39.0            36.6             2.4
73  ALT         1.0        6.0             3.7             2.3
75  WS9         1.0       43.0            41.0             2.0
78  CH1         1.0       19.0            17.1             1.9


## Top 10 Underperforming Teams

In [21]:
bottom10 = MasterDfPart2[MasterDfPart2['Avg. Wins'] < MasterDfPart2['Exp. Avg. Wins']]
print(bottom10.sort_values(by='Avg. Deviation', ascending=True)[:11])

    Team  # of Years  Avg. Wins  Exp. Avg. Wins  Avg. Deviation
149  BSU         1.0       56.0            62.6            -6.6
142  HR1         2.0       35.0            41.3            -6.3
148  IND         1.0       88.0            94.2            -6.2
147  PHN         1.0       14.0            20.0            -6.0
146  CHF         2.0       86.5            91.8            -5.3
145  BFP         1.0       36.0            40.8            -4.8
144  CNU         1.0       69.0            73.7            -4.7
107  BLA         2.0       59.0            63.5            -4.5
143  PHP         1.0       68.0            72.5            -4.5
106  WSN         8.0       51.2            55.7            -4.4
90   CN2         8.0       68.6            73.0            -4.4


# Analysis - Part 3 - Home Field Advantage?
## Now let's see if home field advantage helps teams outperform their expected win/loss totals.
## 

In [22]:
framesPart3 = []
for i in filenames:
    #print(i)
    framesPart3.append(fileReaderPart3(i))

#Part1Master = fileReader('GL2017.txt')
#Part2Master = fileReader('GL1905.txt')
#frames = [Part1Master, Part2Master]
#result = pd.concat(frames).sort_values(by='Deviation', ascending=False)
#result = result.drop_duplicates()


In [23]:
MasterDfPart3 = Part3MasterFrame(framesPart3)
MasterDfPart3 = MasterDfPart3[['Year', 'Team', '# Actual Wins', '# Expected Wins', 'Deviation']]

In [24]:
print(MasterDfPart3)

     Year  Team  # Actual Wins  # Expected Wins  Deviation
0    1871  Home           89.3             91.1       -1.8
1    1871  Away           70.2             70.9       -0.7
2    1872  Home           92.1             94.1       -2.0
3    1872  Away           65.5             67.9       -2.4
4    1873  Home          104.2            106.8       -2.6
..    ...   ...            ...              ...        ...
291  2016  Away           76.1             79.0       -2.9
292  2017  Home           87.4             84.7        2.7
293  2017  Away           74.6             77.3       -2.7
294  2018  Home           85.5             83.8        1.7
295  2018  Away           76.5             78.2       -1.7

[296 rows x 5 columns]
